In [186]:
import numpy as np
import pandas as pd
import nltk
import re,os
os.getcwd()
os.chdir('C://Users//tjoseph//Documents//Python')
df=pd.read_csv('tennis_articles_v4.csv')

In [187]:
len(df)

8

#### Sentence tokenization

In [188]:
from nltk import sent_tokenize
sentences=[]
for i in df['article_text']:
   sentences.append(sent_tokenize(i))
len(sentences)

8

In [189]:
#flatten list
sentences=[y for x in sentences for y in x ]

In [190]:
len(sentences)

119

In [191]:
for i,j in enumerate(sentences):
    if len(j)==0:
        print(i,j)

  ### Extract word_embeddings ,put it in a Dict-keys=word, values=word,co-effs(np.asarray)

In [192]:
word_embeddings={}
f=open('glove.6B.100d.txt',encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coeffs=np.asarray(values[1:],dtype='float32')
    word_embeddings[word]=coeffs
    

In [193]:
len(word_embeddings)

400000

### remove punctuations, numbers and special characters

In [194]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

### make alphabets lowercase

In [195]:
clean_sentences = [s.lower() for s in clean_sentences]

In [196]:
for i,j in enumerate(clean_sentences):
    if len(j)==0:
        print('Empty string found at {0}th index'.format(i),j)

In [197]:
clean_sentences=list(filter(None,clean_sentences))
len(clean_sentences)

119

In [198]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [199]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [200]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [201]:
len(clean_sentences)

119

### Sentence vectors

In [202]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [203]:
sentence_vectors[10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

### Similarity Matrix 

In [204]:
sim_mat=np.zeros([len(sentences),len(sentences)])

In [205]:
sim_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

##### Cosine Similarity to compute the similarity between a pair of sentences

In [221]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), \
                                        sentence_vectors[j].reshape(1,100))[0,0]
      

In [207]:
len(sim_mat)

119

In [212]:
sim_mat[11]

array([0.83178788, 0.8244583 , 0.84927762, 0.91478759, 0.80672944,
       0.82324845, 0.88512778, 0.87815404, 0.88302457, 0.82203555,
       0.        , 0.        , 0.84679693, 0.81501478, 0.95335245,
       0.86245984, 0.83345395, 0.65141165, 0.72773397, 0.76936495,
       0.80212027, 0.54233021, 0.65860659, 0.80001104, 0.72678566,
       0.48568445, 0.67168283, 0.76678622, 0.78344202, 0.81829804,
       0.85870534, 0.79941952, 0.70721054, 0.82291329, 0.81325769,
       0.82108647, 0.66887331, 0.82257837, 0.89703327, 0.81916767,
       0.78287721, 0.8038401 , 0.7498281 , 0.8019082 , 0.82966542,
       0.78515673, 0.80128396, 0.53514379, 0.67698848, 0.74675739,
       0.47783446, 0.73573601, 0.75890696, 0.69260466, 0.81438637,
       0.7085017 , 0.79811317, 0.65863627, 0.79345113, 0.81295437,
       0.72660272, 0.80206501, 0.89252675, 0.85579288, 0.91765726,
       0.73859978, 0.93383825, 0.89497292, 0.93432331, 0.88362312,
       0.80658031, 0.76582584, 0.79009277, 0.86650258, 0.81522

##### Page Rank Algorithm

In [332]:
import networkx as nx
nx_graph=nx.from_numpy_array(sim_mat)
scores=nx.pagerank(nx_graph)

In [340]:
y=max(scores.values())

In [341]:
z=max(scores,key=scores.get)

In [342]:
print('Max value in scores_dictionary :{0} is found at key:{1}'.format(y,z))

Max value in scores_dictionary :0.009293791260564146 is found at key:3


##### Summary Extraction


In [363]:
ranked_sentences=sorted([(scores[i],s) for i,s in enumerate(sentences)],reverse=True)
 

In [368]:
for i in range(5):
    print(ranked_sentences[i][1])

When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.
Currently in ninth place, Nishikori with a win could move to within 125 points of the cut for the eight-man event in London 